# A5 小書建立器（Voila 版，已修正 DOCX 下載）

**特點**
- 使用 `ipywidgets.FileDownload` 正確輸出 **二進位 DOCX**（避免瀏覽器誤存成 HTML）。  
- 支援段落尾端樣式標記：`[14pt]` 等字級設定。  
- 支援圖片標記：`[圖片]`、`[圖片60%]`（百分比指定寬度）。  
- 支援左右雙圖：`[左圖80%][右圖50%]`（以 2 欄表格置入）。  
- 若段落末加上 `[無圖]`，即使該段含 `[圖片]` 也會跳過圖像消耗。  
- 封面／內頁／封底分離，封底不加頁碼；內頁可選擇是否加頁碼。  
- 下載前**自動驗證** DOCX 是否可開。  
  
> **使用方式（Voila）**：  
> 在此 notebook 同目錄內執行：  
> ```bash
> voila A5_Book_App_FIXED.ipynb
> ```
> 或在伺服器環境使用相對應的指令啟動 Voila。


In [ ]:
# ===== 匯入套件 =====
import io, re, math, base64
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from PIL import Image


In [ ]:
# ===== 工具函式 =====

def ensure_rgb_jpeg(img_bytes: bytes, quality=90) -> bytes:
    """將圖片安全轉成 RGB JPEG，避免透明通道/奇怪格式造成 DOCX 損壞。"""
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ("RGB", "L"):
        im = im.convert("RGB")
    out = io.BytesIO()
    im.save(out, format="JPEG", quality=quality)
    return out.getvalue()

def px_to_inches(px: int, dpi: int = 96) -> float:
    return px / dpi

def parse_size_tag(text: str, default_pt: int = 12) -> int:
    """解析段落尾端如 [14pt] 的字級數值；無則回傳 default_pt。"""
    m = re.search(r"\[(\d+)pt\]\s*$", text)
    if m:
        return int(m.group(1))
    return default_pt

def strip_size_tag(text: str) -> str:
    return re.sub(r"\[\d+pt\]\s*$", "", text)

def has_skip_image_tag(text: str) -> bool:
    return bool(re.search(r"\[無圖\]\s*$", text))

def strip_skip_image_tag(text: str) -> str:
    return re.sub(r"\[無圖\]\s*$", "", text)

def find_single_image_tag(text: str):
    """抓取 [圖片] 或 [圖片60%]，回傳 (percent or None, start,end)；若無回傳 None。"""
    m = re.search(r"\[(?:圖片)(\d+)?%?\]", text)
    if not m:
        return None
    pct = m.group(1)
    pct = int(pct) if pct else None
    return pct, m.start(), m.end()

def find_double_image_tags(text: str):
    """抓取 [左圖80%][右圖50%] 類型；回傳 (l_pct, r_pct) 或 None。"""
    ml = re.search(r"\[左圖(\d+)?%?\]", text)
    mr = re.search(r"\[右圖(\d+)?%?\]", text)
    if not (ml and mr):
        return None
    lp = int(ml.group(1)) if ml.group(1) else None
    rp = int(mr.group(1)) if mr.group(1) else None
    return (lp, rp)

def set_section_to_a5(section):
    """設定頁面為 A5，適度邊界。"""
    section.page_width = Mm(148)
    section.page_height = Mm(210)
    section.left_margin = Mm(15)
    section.right_margin = Mm(15)
    section.top_margin = Mm(15)
    section.bottom_margin = Mm(15)

def add_paragraph_with_size(doc, text: str, size_pt: int, align_center=False):
    p = doc.add_paragraph(text)
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER if align_center else WD_ALIGN_PARAGRAPH.LEFT
    for run in p.runs:
        run.font.size = Pt(size_pt)
    return p

def add_single_image_paragraph(doc, img_bytes: bytes, width_pct: int | None = None):
    """插入單圖，寬度可用百分比指定（相對可印刷寬度）。"""
    # 可印刷寬度（A5 寬度 148mm 減左右邊界 30mm ≈ 118mm ≈ 4.65 in）
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    if width_pct is None:
        width_in = page_w_in * 0.9
    else:
        width_in = page_w_in * (width_pct / 100.0)
        width_in = max(1.0, min(page_w_in, width_in))
    safe = ensure_rgb_jpeg(img_bytes)
    doc.add_picture(io.BytesIO(safe), width=Inches(width_in))

def add_double_image_table(doc, left_bytes: bytes, right_bytes: bytes, l_pct: int | None, r_pct: int | None):
    """以 2 欄表格並列左右圖。百分比為相對可印刷寬度。"""
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    # 基本分兩欄，各自再乘上指定比例（給個合理上限）
    base_in = page_w_in / 2.0
    l_in = base_in if l_pct is None else base_in * (l_pct / 100.0) * 1.0
    r_in = base_in if r_pct is None else base_in * (r_pct / 100.0) * 1.0
    l_in = max(1.0, min(page_w_in, l_in))
    r_in = max(1.0, min(page_w_in, r_in))

    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    # 左
    cell = table.rows[0].cells[0]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(left_bytes)), width=Inches(l_in))
    # 右
    cell = table.rows[0].cells[1]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(right_bytes)), width=Inches(r_in))

def add_page_number_footer(section, start_from: int = 1):
    """在頁尾置中加入頁碼，從指定數字開始。"""
    footer = section.footer
    p = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    fld_simple = OxmlElement('w:fldSimple')
    fld_simple.set(qn('w:instr'), 'PAGE')
    r = OxmlElement('w:r')
    t = OxmlElement('w:t')
    t.text = " "  # 讓欄位顯示
    r.append(t)
    fld_simple.append(r)
    p._p.append(fld_simple)

    # 設定編號起始
    sectPr = section._sectPr
    pgNumType = sectPr.xpath('./w:pgNumType')
    if pgNumType:
        pgNumType = pgNumType[0]
    else:
        pgNumType = OxmlElement('w:pgNumType')
        sectPr.append(pgNumType)
    pgNumType.set(qn('w:start'), str(start_from))

def build_docx_bytes(cover_text: str, body_text: str, back_text: str,
                     add_body_page_number: bool,
                     image_bytes_list: list[bytes]) -> bytes:
    """組 DOCX（封面/內頁/封底），並處理文字標記與圖片置入。"""
    doc = Document()
    # Section 1: 封面
    set_section_to_a5(doc.sections[0])
    add_paragraph_with_size(doc, cover_text.strip(), 20, align_center=True)

    # Section 2: 內頁（新節）
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    if add_body_page_number:
        add_page_number_footer(doc.sections[-1], start_from=1)

    # 解析內文：按空行分段
    paragraphs = [p.strip() for p in body_text.split('\n')]
    img_idx = 0

    for raw in paragraphs:
        if not raw:
            doc.add_paragraph("")
            continue

        size_pt = parse_size_tag(raw, default_pt=12)
        text = strip_size_tag(raw)
        skip_img = has_skip_image_tag(text)
        text = strip_skip_image_tag(text)

        # 雙圖？
        dbl = find_double_image_tags(text)
        if dbl and not skip_img and img_idx + 1 < len(image_bytes_list):
            # 去掉 tag 本身的文字
            text_clean = re.sub(r"\[左圖\d*%?\]\s*\[右圖\d*%?\]", "", text)
            if text_clean.strip():
                add_paragraph_with_size(doc, text_clean.strip(), size_pt)
            l_pct, r_pct = dbl
            add_double_image_table(doc, image_bytes_list[img_idx], image_bytes_list[img_idx+1], l_pct, r_pct)
            img_idx += 2
            continue

        # 單圖？
        single = find_single_image_tag(text)
        if single and not skip_img and img_idx < len(image_bytes_list):
            pct, s, e = single
            text_clean = (text[:s] + text[e:]).strip()
            if text_clean:
                add_paragraph_with_size(doc, text_clean, size_pt)
            add_single_image_paragraph(doc, image_bytes_list[img_idx], width_pct=pct)
            img_idx += 1
            continue

        # 純文字
        add_paragraph_with_size(doc, text, size_pt)

    # Section 3: 封底（新節，無頁碼）
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    add_paragraph_with_size(doc, back_text.strip(), 12, align_center=True)

    buf = io.BytesIO()
    doc.save(buf)
    data = buf.getvalue()

    # 驗證：能否重新開啟
    try:
        _ = Document(io.BytesIO(data))
    except Exception as e:
        raise RuntimeError(f"DOCX 驗證失敗：{e}")
    return data


In [ ]:
# ===== 使用者介面 =====

txt_cover = widgets.Textarea(
    value="我的 A5 小書\n（這是封面，會置中）",
    description='封面',
    layout=widgets.Layout(width='100%', height='80px')
)
txt_body = widgets.Textarea(
    value=(
        "這是一段內文。[14pt]\n"
        "這一段想放一張圖：[圖片]\n\n"
        "這一段想放雙圖：[左圖80%][右圖50%]\n"
        "若不想消耗圖片，即使段落含[圖片]，可在尾端加[無圖] 例如：\n"
        "這段含標記但跳過圖像[圖片][無圖]\n"
    ),
    description='內頁',
    layout=widgets.Layout(width='100%', height='180px')
)
txt_back = widgets.Textarea(
    value="封底文字（不加頁碼）",
    description='封底',
    layout=widgets.Layout(width='100%', height='60px')
)

chk_pagenum = widgets.Checkbox(value=True, description='內頁加頁碼')

# 上傳圖片與排序
uploader = widgets.FileUpload(accept='image/*', multiple=True, description='選擇圖片')
listbox = widgets.Select(options=[], rows=6, description='順序')
btn_up = widgets.Button(description='上移')
btn_down = widgets.Button(description='下移')
btn_clear = widgets.Button(description='清空')
order_box = widgets.VBox([listbox, widgets.HBox([btn_up, btn_down, btn_clear])])

def refresh_listbox():
    # 顯示檔名
    names = [meta['name'] for meta in uploader.value]
    listbox.options = names

def move_selected(delta: int):
    if not listbox.options:
        return
    idx = listbox.index
    if idx is None:
        return
    new_idx = idx + delta
    if new_idx < 0 or new_idx >= len(uploader.value):
        return
    # 交換 uploader.value 的順序（list of dict-like）
    vals = list(uploader.value)
    vals[idx], vals[new_idx] = vals[new_idx], vals[idx]
    uploader._counter = len(vals)  # maintain internal consistency
    uploader.value.clear()
    for v in vals:
        uploader.value.append(v)
    refresh_listbox()
    listbox.index = new_idx

def on_upload_change(change):
    refresh_listbox()

uploader.observe(on_upload_change, names='value')
btn_up.on_click(lambda _: move_selected(-1))
btn_down.on_click(lambda _: move_selected(1))
btn_clear.on_click(lambda _: (uploader.value.clear(), refresh_listbox()))

ui = widgets.VBox([
    widgets.HBox([widgets.VBox([txt_cover, txt_back]), widgets.VBox([chk_pagenum])]),
    widgets.Label('—— 內頁 ——'),
    txt_body,
    widgets.Label('—— 圖片上傳與排序（可多選，上下移動調整順序） ——'),
    widgets.HBox([uploader, order_box]),
])
display(ui)


In [ ]:
# ===== 產生 DOCX 並提供下載（正確 MIME；先驗證） =====

def make_docx_bytes_for_download():
    # 依目前 uploader.value 順序取 bytes
    imgs = []
    for meta in uploader.value:
        raw = meta['content']
        # 先不轉，等插入時再轉 RGB JPEG（更保險）
        imgs.append(raw)
    data = build_docx_bytes(
        cover_text=txt_cover.value,
        body_text=txt_body.value,
        back_text=txt_back.value,
        add_body_page_number=chk_pagenum.value,
        image_bytes_list=imgs
    )
    return data

download_btn = widgets.FileDownload(
    data=make_docx_bytes_for_download,
    filename='A5_book_fixed.docx',
    description='⬇️ 下載 DOCX（已驗證）',
    mime='application/vnd.openxmlformats-officedocument.wordprocessingml.document',
)

display(download_btn)
